In [1]:
import os
print(os.getcwd())

/Users/sreeramreddy.thoom/oss-work/forked_uc_ai/unitycatalog/ai/integrations/bedrock


In [2]:
import sys
sys.path.append('./src/unitycatalog/ai/bedrock/')

In [4]:
from toolkit import UCFunctionToolkit

In [5]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

In [6]:
# Set up UC client
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"
api_client = ApiClient(configuration=config)
client = UnitycatalogFunctionClient(api_client=api_client)

In [7]:
# Create toolkit with the weather function
function_name = f"AICatalog.AISchema.location_weather_in_c"
toolkit = UCFunctionToolkit(function_names=[function_name], client=client)

ServiceException: (500)
Reason: Internal Server Error
HTTP response headers: <CIMultiDictProxy('Content-Type': 'application/json', 'Content-Length': '5622', 'Server': 'Armeria/1.28.4', 'Date': 'Fri, 10 Jan 2025 18:58:57 GMT')>
HTTP response body: {"error_code":"INTERNAL","details":[{"reason":"INTERNAL","metadata":{},"@type":"google.rpc.ErrorInfo"}],"stack_trace":"[java.base/java.util.Objects.requireNonNull(Objects.java:235), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:661), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:618), com.linecorp.armeria.common.HttpResponse.ofJson(HttpResponse.java:603), io.unitycatalog.server.service.FunctionService.getFunction(FunctionService.java:99), com.linecorp.armeria.internal.server.annotation.AnnotatedService.invoke(AnnotatedService.java:382), com.linecorp.armeria.internal.server.annotation.AnnotatedService.serve0(AnnotatedService.java:296), com.linecorp.armeria.internal.server.annotation.AnnotatedService.serve